In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('./movies.csv')
ratings = pd.read_csv('./ratings.csv')

In [3]:
ratings['rating'][ratings['rating'] < 4.0] = 0
ratings['rating'][ratings['rating'] >= 4.0] = 1

/home/xtzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/xtzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
def get_first_genre(genres):
    return genres.split('|')[0]

movies['genres'] = movies['genres'].apply(lambda x: get_first_genre(x))

In [7]:
ratings = ratings.merge(movies[['movieId', 'genres']])

In [8]:
ratings.sort_values(by='timestamp',axis=0, ascending=True, inplace=True)

In [9]:
print("Rating Shape: {}".format(ratings.shape))

Rating Shape: (20000263, 5)


In [16]:
ratings.to_csv('./sorted_impl_ratings.csv', index=None, sep='\t')

In [13]:
ratings.head()

,userId,movieId,rating,timestamp,genres
16933064,28507,1176,1.0,789652004,Drama
765628,131160,1079,0.0,789652009,Comedy
7371115,131160,21,0.0,789652009,Comedy
116714,131160,47,1.0,789652009,Mystery
15948094,85252,12,0.0,822873600,Comedy


In [25]:
from collections import Counter
genres_list = movies['genres'].values.tolist()
movie_id_list = movies['movieId'].values.tolist()

genre_counter = Counter()
for genre in genres_list:
    genre_counter.update([genre])

genre_vocab_file = open('./genre_vocab', 'w', encoding='utf8')
movie_id_vocab_file = open('./movie_id_vocab', 'w', encoding='utf8')

movie_id_counter = Counter()
for movie_id in movie_id_list:
    movie_id_counter.update([movie_id])

for genre, genre_cnt in genre_counter.most_common():
    print("{}\t{}".format(genre, genre_cnt), file=genre_vocab_file)

for movie_id, movie_id_cnt in movie_id_counter.most_common():
    print("{}\t{}".format(movie_id, movie_id_cnt), file=movie_id_vocab_file)
    
genre_vocab_file.close()
movie_id_vocab_file.close()

In [26]:
PAD_TOKEN = '<PAD>'
UNKNOWN_TOKEN = '<UNK>'
SENTENCE_START = '<SOS>'
SENTENCE_END = '<EOS>'


class Vocab(object):
    def __init__(
        self,
        vocab_file,
        max_size=10000000,
        use_pad=True,
        use_unk=True,
        use_sos=True,
        use_eos=True
    ):
        self.word2idx = {}
        self.count = 0

        if use_pad:
            self.word2idx[PAD_TOKEN] = self.count
            self.count += 1

        if use_unk:
            self.word2idx[UNKNOWN_TOKEN] = self.count
            self.count += 1

        if use_sos:
            self.word2idx[SENTENCE_START] = self.count
            self.count += 1

        if use_eos:
            self.word2idx[SENTENCE_END] = self.count
            self.count += 1

        with open(vocab_file, 'r', encoding='utf8') as vocab_f:
            for index, line in enumerate(vocab_f):
                items = line.strip().split('\t')
                if len(items) != 2:
                    tf.logging.warn('Warning: wrong fromat in file: %s, line: %s' % (vocab_file, line))
                    pass
                else:
                    word, word_cnt = items[0], items[1]
                    self.word2idx[word] = self.count
                    self.count += 1

                if max_size != 0 and self.count >= max_size:
                    print('max_size of vocab was specified as %i; we now have %i words.Stoping reading.' %
                            (max_size, self.count))
        self.idx2word = dict(list(zip(self.word2idx.values(), self.word2idx.keys())))
        print('Finished constructing vocabulary of %i total words. Last word added: %s' %(self.count,
            self.id2word(self.count - 1)))

    def word2id(self, word):
        if word not in self.word2idx:
            return self.word2idx[UNKNOWN_TOKEN]
        return self.word2idx[word]

    def id2word(self, word_id):
        if word_id not in self.idx2word:
            raise ValueError('Id not found in vocab : %d' % word_id)
        return self.idx2word[word_id]

    def size(self):
        return self.count

In [27]:
genre_vocab = Vocab('./genre_vocab', 100000000, False, False, False, False)
movieid_vocab = Vocab('./movie_id_vocab', 100000000, False, False, False, False)

Finished constructing vocabulary of 20 total words. Last word added: IMAX
Finished constructing vocabulary of 27278 total words. Last word added: 131262


In [30]:
ratings = ratings.drop(['genres'], axis=1)

In [33]:
len(set(ratings['userId'].values.tolist()))

138493